In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
import chromedriver_autoinstaller
import random
from selenium import webdriver
import datetime
import time
import numpy as np
import pandas as pd
import re
from selenium.webdriver.common.keys import Keys 
from urllib.parse import quote_plus
from urllib.request import urlopen
import os

 # Naver food image crawling
 식단관리앱 컨셉에 맞추어 주로 식단 관리하는 분들의 식사 범주에 맞추어 검색어를 선정하였고, 그 외에 일반식 같은 경우에 분류룰 최소화하여 데이터 수집을 진행하였습니다. <br>
 <br>
 크롤링에는 네이버 API를 사용하는 방법과 크롬드라이버를 활용한 셀레니움 라이브러리를 사용하는 방법이 있고, 아래에서는 셀리니움으로 동적 크롤링을 진행하였습니다.
 

## 1. Driver 함수 선언
자동으로 크롬 드라이버의 위치를 찾는 옵션과 크롤링하면서 발생했던 문제 (이미지가 전부 로드 되지않아 제대로 다운이 되지 않았던 현상)를 해결하기 위해서 전체화면으로 실행하도록 설정하였습니다.

In [2]:
def Driver():
    options = webdriver.ChromeOptions()
    options.add_experimental_option("excludeSwitches", ["enable-logging"])
    #options.add_argument("headless")
    options.add_argument('--start-fullscreen')
    options.add_argument('--no-sandbox')
    options.add_argument('--disable-dev-shm-usage')
    wd = webdriver.Chrome(options = options)
    return wd

## 2. 이미지를 수집할 홈페이지 출력
먼저 검색어를 입력할 Base Url과 상업적으로 이용이 가능한 이미지를 필터링할 옵션을 넣어서 홈페이지를 출력하게끔 하였습니다.

In [3]:
# 베이스 URL과 상업적 이용 가능 옵션
def make_url(word):
    base_url = 'https://search.naver.com/search.naver?sm=tab_hty.top&where=image&query='
    ccl = '&res_fr=0&res_to=0&sm=tab_opt&color=&ccl=2&nso=so%3Ar%2Ca%3Aall%2Cp%3Aall&recent=0&datetype=0&startdate=0&enddate=0&gif=0&optStr=&nso_open=1&pq='
    return base_url + word + ccl

## 3. 이미지를 저장할 폴더 생성
검색어를 이용하여 디렉토리를 생성하는 함수입니다.

In [4]:
# 해당하는 폴더가 없을 경우 생성해주는 함수
def makedirs(path): 
   try: 
        os.makedirs(path) 
   except OSError: 
       if not os.path.isdir(path): 
           raise

## 4. 찾을 목록이 담긴 Dictionary
해당 범주에 맞는 음식을 분류하고 추가 삭제가 용이하게 하기 위하여 dictionary를 통해 선언하였습니다.

In [5]:
# 찾을 목록이 담긴 dictionary
search_dict = {
        '건강식단': ['샐러드','고구마요리','닭가슴살','연어음식','오트밀'],
        '밥류' : ['볶음밥','덮밥','한정식','돈까스','오므라이스'],
        '면류' : ['라면','짜장면','짬뽕','파스타','냉면', '국수'],
        '패스트푸드' : ['햄버거','감자튀김','피자','치킨','떢볶이'],
        '디저트류' : ['카페음료','와플','아이스크림','조각케이크','쿠키','빙수']}

## 5. 이미지를 저장
간혹 이미지가 로드되는 경우에는 이미지를 저장하지 않고 넘어가고 그 외에는 음식범주 음식이름 인덱스를 파일명에 추가하여 저장하였습니다.

In [6]:
def save_images(image_url, paths, file_name, i):
    import base64
        
    if 'data:' in str(image_url):
        pass
    else:
        t= urlopen(image_url).read()
        file = open(os.path.join(paths, file_name+'_'+str(i)+".gif"), 'wb')
        file.write(t)

## 6. 이미지 크롤링 진행
이미지에서 다음이미지로 넘어가는 패턴을 XPATH를 통해 찾아내었고 모든 이미지를 로드하기 위한 스크롤 다운 기능과 지연시간을 설정하고 또한 elements를 찾지 못하는 경우 예외처리를 하여 안정적으로 크롤링을 수행할 수 있게 구현하였습니다.

In [7]:
def naver_crawl(image_numbers):
    wd = Driver()
    wd.implicitly_wait(3)
    for meal in search_dict:
        for food in search_dict[meal]:
                # 음식에 해당하는 검색어를 입력한 페이지 출력
                print(f"------------------ Start {meal} / {food} ----------------------")
                wd.get(make_url(food))
                time.sleep(2)
                for i in range(1,image_numbers+1):
                    time.sleep(2)
                    # i에 해당하는 이미지가 없을 경우 PASS
                    try:
                        # image url 추출
                        images= wd.find_elements(By.XPATH, f'//*[@id="main_pack"]/section[2]/div/div[1]/div[1]/div[{i}]/div/div[1]/a/img')
                        save_path = f'/Users/mgd81/CP1/images/{meal}/'
                        makedirs(save_path)
                        src = images[0].get_attribute('src')
                        save_images(str(src), save_path, f'{meal}_{food}_', i)
                        
                        # 이미지가 10개가 넘어갈때 마다 PAGE_DOWN
                        if i % 10 == 0:
                            body = wd.find_element(By.XPATH,'//body').send_keys(Keys.PAGE_DOWN)
                            time.sleep(3)
                    except:
                        print(f"No element in {i}")
                        continue
                print(f"------------------ end {meal} / {food} ----------------------")
    wd.close()
    print("End_crawling")

In [8]:
naver_crawl(45)

------------------ Start 건강식단 / 샐러드 ----------------------
No element in 11
No element in 35
------------------ end 건강식단 / 샐러드 ----------------------
------------------ Start 건강식단 / 고구마요리 ----------------------
No element in 11
No element in 35
------------------ end 건강식단 / 고구마요리 ----------------------
------------------ Start 건강식단 / 닭가슴살 ----------------------
No element in 11
No element in 35
------------------ end 건강식단 / 닭가슴살 ----------------------
------------------ Start 건강식단 / 연어음식 ----------------------
------------------ end 건강식단 / 연어음식 ----------------------
------------------ Start 건강식단 / 오트밀 ----------------------
No element in 11
No element in 35
------------------ end 건강식단 / 오트밀 ----------------------
------------------ Start 밥류 / 볶음밥 ----------------------
No element in 11
No element in 35
------------------ end 밥류 / 볶음밥 ----------------------
------------------ Start 밥류 / 덮밥 ----------------------
No element in 11
No element in 35
------------------ end 밥류 / 덮밥 ---------

## 7. 기대 데이터 결과물 표
|음식의 범주|음식 종류|이미지 개수|
|---|---|---|
|건강 식단|샐러드|300|
||고구마 식단|300|
||닭가슴살 식단|300|
||연어 식단|300|
|밥류|볶음밥|300|
||덮밥|300|
||한정식|300|
||돈까스|300|
||오므라이스|300|
|면류|라면|300|
||짜장면|300|
||짬뽕|300|
||파스타|300|
||냉면|300|
|패스트푸드|햄버거|300|
||감자튀김|300|
||피자|300|
||치킨|300|
|디저트류|카페음료|300|
||와플|300|
||아이스크림|300|
||탄산음료|300|
||조각케이크|300|
||쿠키|300|
||빙수|300|
